In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, TimestampType, ArrayType
from pyspark.sql.functions import col, to_date, hour, count, lit, struct, explode

In [ ]:
def define_schema():
    ad_campaigns_schema = StructType([
    StructField("campaign_id", StringType(), True),
    StructField("campaign_name", StringType(), True),
    StructField("campaign_country", StringType(), True),
    StructField("os_type", StringType(), True),
    StructField("device_type", StringType(), True),
    StructField("place_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("event_type", StringType(), True),
    StructField("event_time", TimestampType(), True)  # since it’s an ISO datetime
    ])

    user_profile_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("country", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("age_group", StringType(), True),
    StructField("category", ArrayType(StringType()), True)  # array of strings
    ])

    store_schema = StructType([
    StructField("store_name", StringType(), True),
    StructField("place_ids", ArrayType(StringType()), True)  # array of strings
    ])
    return ad_campaigns_schema, user_profile_schema, store_schema

In [ ]:
def load_data(spark, ad_campaigns_schema, user_profile_schema, store_schema):
    adc_df= spark.read.option("multiline", "true").schema(ad_campaigns_schema).json("/projects/marketingDA/input_data/ad_campaigns_data.json")
    up_df = spark.read.option("multiline", "true").schema(user_profile_schema).json("/projects/marketingDA/input_data/user_profile_data.json")
    s_df = spark.read.option("multiline", "true").schema(store_schema).json("/projects/marketingDA/input_data/store_data.json")
    return adc_df, up_df, s_df

In [ ]:
def exploding_df_q1(up_df, s_df):
    up_df= up_df.withColumn("category", explode("category"))
    s_df= s_df.withColumn("place_ids", explode("place_ids"))
    return up_df, s_df

In [ ]:
def sol_q1(adc_df):
    adc_df_temp= adc_df.select("campaign_id", "os_type", "event_type", to_date(col("event_time")).alias("date"), hour(col("event_time")) \
        .alias("hour"))
    grouped_df= adc_df_temp.groupBy("campaign_id", "date", "hour", "os_type", "event_type") \
            .agg(count("*").alias("event_count"))
    grouped_df= grouped_df.withColumnRenamed("os_type", "value").withColumn("type", lit("os_type")) \
            .select("campaign_id", "date", "hour", "type", "value", "event_type", "event_count")
    pivoted_df= grouped_df.groupBy("campaign_id", "date", "hour", "type", "value").pivot("event_type").sum("event_count")
    struct_df= pivoted_df.select("campaign_id", "date", "hour", "type", "value", \
                            struct(col("impression"), col("click"), col("video ad")).alias("event"))
    return struct_df

In [ ]:
def sol_q2(adc_df, s_df):
    joined_df= adc_df.join(s_df,  adc_df.place_id == s_df.place_ids, "inner")
    q2_joined_df= joined_df.select("campaign_id", to_date(col("event_time")).alias("date"), hour(col("event_time")) \
                 .alias("hour"),"store_name", "event_type") \
                         .withColumn("type", lit("store_name")).withColumnRenamed("store_name", "value")
    q2_agg_df= q2_joined_df.groupBy("campaign_id", "date", "hour", "type", "value", "event_type").agg(count("*").alias("event_count"))
    q2_pivoted_df= q2_agg_df.groupBy("campaign_id", "date", "hour", "type", "value").pivot("event_type").sum("event_count").fillna(0)
    q2_struct_df= q2_pivoted_df.select( \
                                  "campaign_id", "date", "hour", "type", "value", \
                                  struct(col("impression"), col("click"), col("video ad")).alias("event") \
                                  )
    return q2_struct_df

In [ ]:
def sol_q3(adc_df, up_df):
    new_up_df= up_df.select("user_id", "gender").distinct()
    new_adc_df= adc_df.select("campaign_id", to_date(col("event_time")).alias("date"), hour(col("event_time")).alias("hour"), "user_id", \
              "event_type").withColumn("type", lit("gender"))
    q3_joined_df= new_adc_df.join(new_up_df, new_adc_df.user_id== new_up_df.user_id, "inner") \
              .select("campaign_id", "date", "hour", "type", "gender", "event_type").withColumnRenamed("gender", "value")
    resultant_df_q3= q3_joined_df.groupBy("campaign_id", "date", "hour", "type", "value", "event_type") \
                 .agg(count("*").alias("event_count"))
    q3_pivoted_df= resultant_df_q3.groupBy("campaign_id", "date", "hour", "type", "value") \
            .pivot("event_type").sum("event_count").fillna(0)
    q3_struct_df= q3_pivoted_df.select("campaign_id", "date", "hour", "type", "value", \
                                   struct(col("impression"), col("click"), col("video ad")).alias("event"))
    return q3_struct_df

In [ ]:
if __name__== "__main__":
    spark= SparkSession.builder \
       .appName("Marketing Campaign DA") \
       .enableHiveSupport() \
       .getOrCreate()
    schema= define_schema()
    adc_df, up_df, s_df= load_data(spark, schema[0], schema[1], schema[2])
    up_df, s_df= exploding_df_q1(up_df, s_df)
    final_df_q1= sol_q1(adc_df)
    final_df_q1.write.mode("overwrite").json("/projects/marketingDA/output_data/marketing_DA/ques_1")
    final_df_q2= sol_q2(adc_df, s_df)
    final_df_q2.write.mode("overwrite").json("/projects/marketingDA/output_data/marketing_DA/ques_2")
    final_df_q3= sol_q3(adc_df, up_df)
    final_df_q3.write.mode("overwrite").json("/projects/marketingDA/output_data/marketing_DA/ques_3")